# Setup

In [1]:
%%capture
! pip install datasets metrics transformers wandb

In [2]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from transformers import BertTokenizerFast, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import BertModel, BertTokenizer
from transformers.file_utils import ModelOutput
from datasets import Dataset
from torch import nn
import pandas as pd
import torch
import gc

# Custom BERT model

In [11]:
class BERTDeepFC(nn.Module):

    def __init__(self,
                 model_name='bert-base-uncased',
                 device='cuda',
                 num_classes=3):
        super(BERTDeepFC, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.leaky_relu = nn.LeakyReLU()
        self.fc1 = nn.Linear(768, 768).to(device)
        self.fc2 = nn.Linear(768, 768).to(device)
        self.fc3 = nn.Linear(768, num_classes).to(device)  # (BERT CLS size, num_classes)
        self.softmax = nn.LogSoftmax(dim=1).to(device)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            output_hidden_states=True)
        x = self.fc1(outputs[0][:, 0, :])
        x = self.dropout(x)
        x = self.leaky_relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.leaky_relu(x)
        x = self.fc3(x)
        c = self.softmax(x)

        # Clean cache
        gc.collect()
        torch.cuda.empty_cache()
        del outputs

        # Compute loss
        loss = None
        if labels is not None:
            ce_loss = nn.CrossEntropyLoss()
            loss = ce_loss(c, labels)

        return ModelOutput({
            'loss': loss,
            'last_hidden_state': c
        })

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
model = BERTDeepFC(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Pre-procesing

In [8]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels

    inputs = tokenizer(
        batch["1"],
        max_length=50,
        padding="max_length",
        truncation=True,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    labels = list(map(lambda x: int(x.split('__label__')[1]), batch['0']))
    batch["label"] = labels

    return batch

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate f1 using sklearn's function
  f_score = f1_score(labels, preds, pos_label=0, average='binary')
  accuracy = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, pos_label=0, average='binary')
  recall = recall_score(labels, preds, pos_label=0, average='binary')
  return {
      'f1_score': f_score,
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall
  }
    
tokenize_batch_size = 32768
df = pd.read_csv('/content/drive/MyDrive/PoliTo/Deep Natural Language Procesing/Project/labeledEligibilitySample1000000.csv', sep='\t', header=None)

In [9]:
train_ds = Dataset.from_pandas(df)

train_ds = train_ds.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=tokenize_batch_size,
    remove_columns=["0", "1"]
)

train_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

  0%|          | 0/31 [00:00<?, ?ba/s]

In [13]:
train_ds.to_csv('tokenized_CTC.csv')

Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

419889032

In [12]:
train_testvalid = train_ds.train_test_split(train_size=0.9)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# Training and evaluation

In [14]:
training_args = TrainingArguments(
    output_dir='./DeepFC_CTC_results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=128,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    fp16=True,                       # enable fp16 apex training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    report_to="wandb"
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_testvalid['train'],         # training dataset
    eval_dataset=test_valid['train'],          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

trainer.train()

preds = trainer.predict(test_valid['test'])

preds[2]

Using amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 900000
  Num Epochs = 2
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 14064
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,F1 Score,Accuracy,Precision,Recall
400,0.519400,0.308730,0.872589,0.878340,0.914118,0.834669
800,0.286600,0.279920,0.890889,0.895740,0.932562,0.852781
1200,0.247200,0.233225,0.909455,0.911320,0.927292,0.892290
1600,0.233700,0.223125,0.913185,0.915960,0.942591,0.885559
2000,0.225400,0.210084,0.922367,0.923300,0.932046,0.912887
2400,0.213800,0.193788,0.926965,0.926940,0.925023,0.928915
2800,0.205100,0.191382,0.926927,0.927820,0.936848,0.917214
3200,0.198600,0.183838,0.930737,0.931200,0.935370,0.926150
3600,0.190400,0.176338,0.933597,0.934200,0.940545,0.926751
4000,0.184600,0.175927,0.934936,0.934520,0.927417,0.942579


***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./DeepFC_CTC_results/checkpoint-400
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./DeepFC_CTC_results/checkpoint-800
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./DeepFC_CTC_results/checkpoint-1200
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./DeepFC_CTC_results/checkpoint-1600
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./DeepFC_CTC_results/checkpoint-2000
Trainer.model is not a `PreTrainedMod

{'test_accuracy': 0.95494,
 'test_f1_score': 0.9546196144782162,
 'test_loss': 0.12443411350250244,
 'test_precision': 0.9608709755899765,
 'test_recall': 0.948449069441665,
 'test_runtime': 116.4029,
 'test_samples_per_second': 429.543,
 'test_steps_per_second': 3.359}

In [16]:
import torch
torch.save(trainer.model, '/content/drive/MyDrive/PoliTo/Deep Natural Language Procesing/Project/model.model')